In [1]:
from pathlib import Path

import pandas as pd

In [2]:
data_dir = Path.cwd().parent / "data"

df = pd.read_csv(data_dir / "factors.csv")


In [ ]:
df

,PERMNO,date,tic,conm,cusip,market_cap,n_shares,n_months,ret_arith,ret_geo,vol_36m,value,profitability,investment
0,10006,1962,4165A,ACF INDUSTRIES INC,00099V004,98804.0,1453.0,12,0.018009,0.014739,NaN,804.042837,0.126949,NaN
1,10006,1963,4165A,ACF INDUSTRIES INC,00099V004,146026.5,2906.0,12,0.045897,0.042782,NaN,1026.613283,0.132943,0.983555
2,10006,1964,4165A,ACF INDUSTRIES INC,00099V004,256476.0,2948.0,12,0.024327,0.023460,0.064884,1650.860266,0.191878,0.936904
3,10006,1965,4165A,ACF INDUSTRIES INC,00099V004,265636.0,5936.0,12,0.019789,0.018039,0.056150,1539.374482,0.258517,0.928158
4,10006,1966,4165A,ACF INDUSTRIES INC,00099V004,233100.0,5920.0,12,-0.012097,-0.013576,0.059158,1231.196653,0.252736,0.893520
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45924,93436,2020,TSLA,TESLA INC,88160R101,200844660.0,186000.0,12,0.229792,0.194458,0.250759,30348.241160,0.012088,0.866817
45925,93436,2021,TSLA,TESLA INC,88160R101,668826839.1,984003.0,12,0.043175,0.034225,0.252244,29890.366424,0.097605,0.657916
45926,93436,2022,TSLA,TESLA INC,88160R101,701030220.0,1041000.0,12,-0.068295,-0.083836,0.253693,23202.933174,0.215007,0.839323
45927,93436,2023,TSLA,TESLA INC,88160R101,830857980.0,3174000.0,12,0.075691,0.060220,0.157126,18551.734471,0.308847,0.754585
